In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import requests
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FYP/Generated Datasets/End of Season/train_test_dataset.csv')
df.head()

,season_x,element,name,position,GW,gw_points,was_home,minutes,minutes_per_gameweek,prev_game_minutes_1,...,previous_season_yellow_cards_per_game,previous_season_red_cards_per_game,opp_team_goals_scored_per_game,opp_team_xG_per_game,opp_team_goals_conceded_per_game,opp_team_xGA_per_game,opp_team_goals_scored_form,opp_team_goals_conceded_form,opp_team_goalless_games_per_game,opp_team_clean_sheets_per_game
0,2017-18,443,Aaron Cresswell,DEF,1,0,0,9,0.00,0.0,...,0.076923,0.038462,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,2017-18,443,Aaron Cresswell,DEF,2,1,0,90,9.00,9.0,...,0.076923,0.038462,0.000000,2.000000,0.0,0.300000,0.000000,0.0,1.000000,1.0
2,2017-18,443,Aaron Cresswell,DEF,3,1,0,90,49.50,90.0,...,0.076923,0.038462,0.000000,0.750000,1.5,1.400000,0.000000,1.5,1.000000,0.0
3,2017-18,443,Aaron Cresswell,DEF,4,6,1,90,63.00,90.0,...,0.076923,0.038462,1.333333,1.066667,0.0,0.833333,1.333333,0.0,0.333333,1.0
4,2017-18,443,Aaron Cresswell,DEF,5,9,0,90,69.75,90.0,...,0.076923,0.038462,1.000000,1.000000,1.0,0.875000,1.000000,1.0,0.000000,0.5


In [103]:
df = df.loc[(df['season_x'] != '2020-21')]

In [104]:
df

,season_x,element,name,position,GW,gw_points,was_home,minutes,minutes_per_gameweek,prev_game_minutes_1,...,previous_season_yellow_cards_per_game,previous_season_red_cards_per_game,opp_team_goals_scored_per_game,opp_team_xG_per_game,opp_team_goals_conceded_per_game,opp_team_xGA_per_game,opp_team_goals_scored_form,opp_team_goals_conceded_form,opp_team_goalless_games_per_game,opp_team_clean_sheets_per_game
0,2017-18,443,Aaron Cresswell,DEF,1,0,0,9,0.000000,0.0,...,0.076923,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,2017-18,443,Aaron Cresswell,DEF,2,1,0,90,9.000000,9.0,...,0.076923,0.038462,0.000000,2.000000,0.000000,0.300000,0.000000,0.0,1.000000,1.000000
2,2017-18,443,Aaron Cresswell,DEF,3,1,0,90,49.500000,90.0,...,0.076923,0.038462,0.000000,0.750000,1.500000,1.400000,0.000000,1.5,1.000000,0.000000
3,2017-18,443,Aaron Cresswell,DEF,4,6,1,90,63.000000,90.0,...,0.076923,0.038462,1.333333,1.066667,0.000000,0.833333,1.333333,0.0,0.333333,1.000000
4,2017-18,443,Aaron Cresswell,DEF,5,9,0,90,69.750000,90.0,...,0.076923,0.038462,1.000000,1.000000,1.000000,0.875000,1.000000,1.0,0.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172854,2023-24,594,Zeki Amdouni,FWD,34,1,0,5,55.575758,0.0,...,0.065160,0.002971,0.937500,0.940625,2.625000,1.925000,1.600000,2.4,0.406250,0.031250
172855,2023-24,594,Zeki Amdouni,FWD,35,7,0,33,54.088235,5.0,...,0.065160,0.002971,1.545455,1.521212,1.515152,1.809091,2.400000,2.2,0.212121,0.242424
172856,2023-24,594,Zeki Amdouni,FWD,36,1,1,5,53.485714,33.0,...,0.065160,0.002971,2.176471,1.908824,1.617647,1.661765,2.200000,0.8,0.147059,0.294118
172857,2023-24,594,Zeki Amdouni,FWD,37,1,0,2,52.138889,5.0,...,0.065160,0.002971,1.971429,1.728571,1.657143,1.697143,1.400000,2.8,0.085714,0.171429


In [105]:
df = df.drop(['element', 'name', 'minutes'], axis=1)

# Splitting DF by Position

df_GKP = df[df.position == 'GKP']
df_DEF = df[df.position == 'DEF']
df_MID = df[df.position == 'MID']
df_FWD = df[df.position == 'FWD']

# Removing Position

df_GKP.drop('position', axis=1, inplace=True)
df_DEF.drop(columns=['position', 'saves_per_game', 'previous_season_saves_per_game'], axis=1, inplace=True)
df_MID.drop(columns=['position', 'saves_per_game', 'previous_season_saves_per_game'], axis=1, inplace=True)
df_FWD.drop(columns=['position', 'saves_per_game', 'previous_season_saves_per_game', 'clean_sheets_per_game', 'previous_season_clean_sheets_per_game'], axis=1, inplace=True)

<ipython-input-105-18a24fd27212>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GKP.drop('position', axis=1, inplace=True)
<ipython-input-105-18a24fd27212>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_DEF.drop(columns=['position', 'saves_per_game', 'previous_season_saves_per_game'], axis=1, inplace=True)
<ipython-input-105-18a24fd27212>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MID.drop(columns=['position', 'sa

In [106]:
with open('/content/drive/MyDrive/Colab Notebooks/FYP/Scalers/gkp_scaler.pkl', 'rb') as f:
  scaler_GKP = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/FYP/Scalers/def_scaler.pkl', 'rb') as f:
  scaler_DEF = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/FYP/Scalers/mid_scaler.pkl', 'rb') as f:
  scaler_MID = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/FYP/Scalers/fwd_scaler.pkl', 'rb') as f:
  scaler_FWD = pickle.load(f)

In [107]:
X = df_GKP.drop(columns=['gw_points'])
y = df_GKP[['season_x', 'gw_points']]

X_train_GKP = X[X.season_x != '2023-24']
X_test_GKP = X[X.season_x == '2023-24']
y_train_GKP = y[y.season_x != '2023-24']
y_test_GKP = y[X.season_x == '2023-24']

X_train_GKP = X_train_GKP.drop(columns=['season_x'])
X_test_GKP = X_test_GKP.drop(columns=['season_x'])
y_train_GKP = y_train_GKP.drop(columns=['season_x'])
y_test_GKP = y_test_GKP.drop(columns=['season_x'])

X_train_GKP = scaler_GKP.fit_transform(X_train_GKP)
X_test_GKP = scaler_GKP.transform(X_test_GKP)
y_train_GKP = np.array(y_train_GKP).reshape(-1)
y_test_GKP = np.array(y_test_GKP).reshape(-1)

In [108]:
X = df_DEF.drop(columns=['gw_points'])
y = df_DEF[['season_x', 'gw_points']]

X_train_DEF = X[X.season_x != '2023-24']
X_test_DEF = X[X.season_x == '2023-24']
y_train_DEF = y[y.season_x != '2023-24']
y_test_DEF = y[X.season_x == '2023-24']

X_train_DEF = X_train_DEF.drop(columns=['season_x'])
X_test_DEF = X_test_DEF.drop(columns=['season_x'])
y_train_DEF = y_train_DEF.drop(columns=['season_x'])
y_test_DEF = y_test_DEF.drop(columns=['season_x'])

X_train_DEF = scaler_DEF.fit_transform(X_train_DEF)
X_test_DEF = scaler_DEF.transform(X_test_DEF)
y_train_DEF = np.array(y_train_DEF).reshape(-1)
y_test_DEF = np.array(y_test_DEF).reshape(-1)

In [109]:
X = df_MID.drop(columns=['gw_points'])
y = df_MID[['season_x', 'gw_points']]

X_train_MID = X[X.season_x != '2023-24']
X_test_MID = X[X.season_x == '2023-24']
y_train_MID = y[y.season_x != '2023-24']
y_test_MID = y[X.season_x == '2023-24']

X_train_MID = X_train_MID.drop(columns=['season_x'])
X_test_MID = X_test_MID.drop(columns=['season_x'])
y_train_MID = y_train_MID.drop(columns=['season_x'])
y_test_MID = y_test_MID.drop(columns=['season_x'])

X_train_MID = scaler_MID.fit_transform(X_train_MID)
X_test_MID = scaler_MID.transform(X_test_MID)
y_train_MID = np.array(y_train_MID).reshape(-1)
y_test_MID = np.array(y_test_MID).reshape(-1)

In [110]:
X = df_FWD.drop(columns=['gw_points'])
y = df_FWD[['season_x', 'gw_points']]

X_train_FWD = X[X.season_x != '2023-24']
X_test_FWD = X[X.season_x == '2023-24']
y_train_FWD = y[y.season_x != '2023-24']
y_test_FWD = y[X.season_x == '2023-24']

X_train_FWD = X_train_FWD.drop(columns=['season_x'])
X_test_FWD = X_test_FWD.drop(columns=['season_x'])
y_train_FWD = y_train_FWD.drop(columns=['season_x'])
y_test_FWD = y_test_FWD.drop(columns=['season_x'])

X_train_FWD = scaler_FWD.fit_transform(X_train_FWD)
X_test_FWD = scaler_FWD.transform(X_test_FWD)
y_train_FWD = np.array(y_train_FWD).reshape(-1)
y_test_FWD = np.array(y_test_FWD).reshape(-1)

In [111]:
model = GradientBoostingRegressor(random_state=0)

model.fit(X_train_GKP, y_train_GKP)
y_pred = model.predict(X_test_GKP)
print(mean_absolute_error(y_test_GKP, y_pred))
print(mean_squared_error(y_test_GKP, y_pred))

0.6281846464482891
2.0213225950656595


In [112]:
model = GradientBoostingRegressor(random_state=0)

model.fit(X_train_DEF, y_train_DEF)
y_pred = model.predict(X_test_DEF)
print(mean_absolute_error(y_test_DEF, y_pred))
print(mean_squared_error(y_test_DEF, y_pred))

1.0297806721508487
3.745693596706305


In [113]:
model = GradientBoostingRegressor(random_state=0)

model.fit(X_train_MID, y_train_MID)
y_pred = model.predict(X_test_MID)
print(mean_absolute_error(y_test_MID, y_pred))
print(mean_squared_error(y_test_MID, y_pred))

0.9385772700584261
3.7810815050305115


In [114]:
model = GradientBoostingRegressor(random_state=0)

model.fit(X_train_FWD, y_train_FWD)
y_pred = model.predict(X_test_FWD)
print(mean_absolute_error(y_test_FWD, y_pred))
print(mean_squared_error(y_test_FWD, y_pred))

1.058598896184476
4.533025385302791
